In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data.describe()

In [ ]:
data = data.drop_duplicates()
data.describe(include=['object'])

In [ ]:
data.info()

In [ ]:
# data = data.drop(columns="Cabin")
# data['Age'] = data['Age'].fillna(data['Age'].median())
# data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

# data.info()
data.head()

In [ ]:
# Usando apply con una función lambda
data['Sex'] = data['Sex'].apply(lambda x: 1 if x == 'female' else 0).astype(int)
# data['Sex'] = data['Sex'].map({'female': 1, 'male': 0}).astype(int)
data.info()

In [ ]:
data.head()

In [136]:
import pandas as pd

Q1 = data['Age'].quantile(0.25)  # Primer cuartil (25%)
Q3 = data['Age'].quantile(0.75)  # Tercer cuartil (75%)
IQR = Q3 - Q1  

lower_bound = Q1 - 1.5 * IQR  
upper_bound = Q3 + 1.5 * IQR  

median_age = data['Age'].median()
data['Age'] = data['Age'].apply(lambda x: median_age if x < lower_bound or x > upper_bound else x)

print(data['Age'].describe())
data.info()

count    891.000000
mean      28.476992
std        9.793559
min        3.000000
25%       23.750000
50%       28.000000
75%       33.000000
max       54.000000
Name: Age, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PassengerId       891 non-null    int64  
 1   Survived          891 non-null    int64  
 2   Pclass            891 non-null    int64  
 3   Name              891 non-null    object 
 4   Sex               891 non-null    int64  
 5   Age               891 non-null    float64
 6   SibSp             891 non-null    int64  
 7   Parch             891 non-null    int64  
 8   Ticket            891 non-null    object 
 9   Fare              891 non-null    float64
 10  Embarked          891 non-null    object 
 11  Embarked_encoded  891 non-null    float64
 12  FlagSolo          891 non-null    int64  
dtyp

In [ ]:
# data.head()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
data['Embarked_encoded'] = encoder.fit_transform(data[['Embarked']])
data.head()

In [ ]:
import pandas as pd

# Seleccionar solo las columnas numéricas
numeric_columns = data.select_dtypes(include=['number'])


numeric_column_names = numeric_columns.columns.tolist()
print("Numeric Column Names:", numeric_column_names)



# EDA

In [ ]:
survival_grouped_by_sex = data.groupby(by=['Survived', 'Sex']).count().unstack(level=0)['PassengerId']
print(survival_grouped_by_sex)
survival_grouped_by_sex.plot.bar()
survival_grouped_by_sex_1 = data.groupby(by=['Survived', 'Sex']).count()['PassengerId']
survival_grouped_by_sex_1

In [ ]:
sns.barplot(x="Pclass", y='Survived', data=data, hue=data['Sex'])
plt.show()
sns.barplot(x="Pclass", y='Survived', data=data, hue=data['Pclass'])
plt.show()

In [ ]:
sns.barplot(x="SibSp", y='Survived', data=data, hue=data['Sex'])

In [ ]:
sns.barplot(x="Parch", y='Survived', data=data, hue=data['Sex'])

In [ ]:
import numpy as np
data['FlagSolo'] = np.where(
    (data['SibSp'] == 0) & (data['Parch'] == 0), 1, 0)

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data[data['Survived'] == 1]['Age'], bins=30, kde=True, color="blue", label="Survived = 1", alpha=0.6)
plt.title("Distribución de Edad para Supervivientes")
plt.xlabel("Edad")
plt.ylabel("Frecuencia")
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(data[data['Survived'] == 0]['Age'], bins=30, kde=True, color="red", label="Survived = 0", alpha=0.6)
plt.title("Distribución de Edad para No Supervivientes")
plt.xlabel("Edad")
plt.ylabel("Frecuencia")
plt.legend()
plt.show()

In [ ]:
age_mean = data.groupby("Survived")["Age"].mean()

plt.figure(figsize=(6, 4))
age_mean.plot(kind='bar', color=["red", "blue"], alpha=0.7)
plt.title("Promedio de Edad por Supervivencia")
plt.xlabel("Survived (0 = No, 1 = Sí)")
plt.ylabel("Edad Promedio")
plt.show()

In [ ]:
import pandas as pd

# Seleccionar solo las columnas numéricas
numeric_columns = data.select_dtypes(include=['number'])


numeric_column_names = numeric_columns.columns.tolist()
print("Numeric Column Names:", numeric_column_names)

In [ ]:
# Calcular la matriz de correlación
correlation_matrix = data[numeric_column_names].corr()

# Mostrar la matriz de correlación

# Configurar el tamaño del gráfico
plt.figure(figsize=(10, 8))

# Crear el heatmap
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)

# Añadir título
plt.title("Heatmap de Correlación entre Variables")
plt.show()

# Calculate the correlation of all features with the target variable 'MedHouseVal'
correlation_with_target = data[numeric_column_names].corr()['Survived'].abs().sort_values(ascending=False)

# Display the correlations
print(correlation_with_target)

In [137]:
from scipy.stats import ttest_ind

group_survived = data[data['Survived'] == 1]['Age'].dropna()
group_not_survived = data[data['Survived'] == 0]['Age'].dropna()

t_stat, p_value = ttest_ind(group_survived, group_not_survived)
print(f"T-statistic: {t_stat}, P-value: {p_value}")

if p_value < 0.05:
    print("Age and Survived relationed")
else:
    print("Age and Survived not relationed")

T-statistic: -0.6091447105050983, P-value: 0.5425841552024884
Age and Survived not relationed


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Select relevant features for VIF calculation
X = data[numeric_column_names]
X = X.drop(columns=["PassengerId", "Survived", "Parch", "SibSp", "Age"])

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data.sort_values(by="VIF", ascending=False))

# No existe multicolinealidad en exceso // Mantenemos las caracteristicas 

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
import numpy as np

target = data['Survived']
# selected_features = ['Pclass', 'Sex', 'Age', 'FlagSolo', 'Fare', 'Embarked_encoded']

data_selected = data[numeric_column_names]


# Filter Method for Feature Selection
selector = SelectKBest(score_func=f_classif, k=5)
selected_features = selector.fit_transform(data_selected, target)

print("Selected Features Shape:", selected_features.shape)
selected_feature_names = data_selected.columns[selector.get_support()]
print("Selected Feature Names:", selected_feature_names)
data[selected_feature_names].head()

In [ ]:
class_counts = data['Survived'].value_counts()

class_counts.plot(kind='bar', color=['blue', 'orange'])
plt.title("Balance de clases en 'Survived'")
plt.xlabel("Clase")
plt.ylabel("Número de instancias")
plt.xticks(ticks=[0, 1], labels=['No sobrevivió', 'Sobrevivió'], rotation=0)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

target = data['Survived']
X = data[selected_feature_names].drop(columns='Survived')

X_train, X_val, y_train, y_val = train_test_split(
    X, target, test_size=0.2, random_state=42
)

print(y_train.shape, X_train.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Crear el Pipeline con un paso genérico para el modelo
pipeline = Pipeline([
    ('model', RandomForestClassifier(random_state=42))  # Este será sustituido en el GridSearch
])

# Definir los hiperparámetros para cada modelo
param_grid = [
    {
        'model': [RandomForestClassifier(random_state=42, class_weight="balanced")],
        'model__n_estimators': [10, 50, 150, 200],
        'model__max_depth': [None, 10, 50, 150],
        'model__max_leaf_nodes': [10, 30, 50, 70],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2, 3, 4],
        'model__criterion': ['gini', 'entropy']
    },
    {
        'model': [LogisticRegression(random_state=42, max_iter=500)],
        'model__C': [0.1, 1, 10],
        'model__penalty': ['l2'],
        'model__solver': ['lbfgs', 'liblinear']
    },
    {
        'model': [SVC(random_state=42)],
        'model__C': [0.1, 1, 10],
        'model__kernel': ['linear', 'rbf'],
        'model__gamma': ['scale', 'auto']
    },
    {
        'model': [DecisionTreeClassifier(random_state=42)],
        'model__max_depth': [10, 50, 150, 200],
        'model__max_leaf_nodes': [10, 30, 50, 70],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2, 3, 4],
        'model__criterion': ['gini', 'entropy']
    }
]

# Configurar el GridSearchCV
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,  # Validación cruzada
    scoring='accuracy',  # Métrica a optimizar
    verbose=2,
    n_jobs=-1  # Usar todos los núcleos disponibles
)

# Ejecutar el GridSearch
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros y el modelo ganador
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
print("Best Model:", grid_search.best_estimator_)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, recall_score, f1_score, precision_score
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_val)

confusion_matrix = confusion_matrix(y_val, y_pred)

# Mostrar la matriz de confusión con visualización
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix, display_labels=best_model.classes_)
disp.plot(cmap=plt.cm.Blues, values_format="d")

accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='binary')
recall = recall_score(y_val, y_pred, average='binary')
f1 = f1_score(y_val, y_pred, average='binary')

# Mostrar las métricas
print(f"{best_model}")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Predicciones

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
selected_features = ['Pclass', 'Sex', 'Fare', 'FlagSolo']




encoder = OrdinalEncoder()
# ahora hay que preparar el test set para evaluación

test = pd.read_csv('test.csv')
# preprocesando test set
test['Embarked_encoded'] = encoder.fit_transform(test[['Embarked']])

# hacer map a Sex
test['Sex'] = test['Sex'].map({'female': 1, 'male': 0}).astype(int)

# rellenar Age
test['Age'] = test['Age'].fillna(28.0)

# Crear FlagSolo
test['FlagSolo'] = np.where(
    (test['SibSp'] == 0) & (test['Parch'] == 0), 1, 0)
test[selected_features].head(3)
# crear test set 
X_test = test[selected_features]

print(X_test.shape)
print(X_test.head())

# prediccion de Survived en test set
Y_pred_rf = best_model.predict(X_test)

In [ ]:
# prediciendo sobre el test set
print(Y_pred_rf[0:20])
# para descargar en ordenador
def download_output(y_pred, name):
  output = pd.DataFrame({'PassengerId': test.PassengerId, 
                         'Survived': y_pred})
  output.to_csv(name, index=False)
download_output(Y_pred_rf, 'ml_prediction.csv')